In [4]:
import pandas as pd
import os
import itertools
import numpy as np

In [87]:
class Market:
    def __init__(self, currencies, dataPath, referenceCurrency='USD', 
                 initialValue=1000, transactionFee=0.005, initialTime=None, timeframe=50):
        self.path = dataPath
        self.currencies = []
        self.value = initialValue
        self.reference = referenceCurrency
        self.fee = transactionFee
        self.time = initialTime #Fill in
        self.currencies = (currencies)
        self.currencies.remove(self.reference)
        self.currencies.insert(0, self.reference)
        self.m = len(self.currencies)
        self.majorPairs = ['EURUSD', 'GBPUSD']
        self.portfolio = None
        pairs = list(itertools.permutations(currencies, 2))
        self.df = self.importFile(pairs)
        self.timeframe = timeframe
        
    def importFile(self, pairs):
        df = {}
        dataPath = self.path
        availableFiles = os.listdir(dataPath)
        for pairTuple in pairs:
            pair = pairTuple[0] + pairTuple[1]
            if (pair + '.csv') in availableFiles:
                if dataPath.endswith('/'):
                    if os.path.isfile(dataPath + pair + '.csv'):
                        df[pair] = pd.read_csv(dataPath + pair + '.csv', delimiter='\t', 
                                               usecols=['Timestamp', 'Open', 'High', 'Low', 'Close'])
                    else:
                        continue
                else:
                    if os.path.isfile(dataPath + '/' + pair + '.csv'):
                        df[pair] = (pd.read_csv(dataPath + '/' + pair + '.csv', delimiter='\t', 
                                                usecols=['Timestamp', 'Open', 'High', 'Low', 'Close']))
                    else:
                        continue
        return df
    
    def getPortfolioSize(self):
        return self.m
    
    def getPairs(self):
        return self.pairs
    
    def initPortfolio(self):
        np.random.seed(5)
        self.portfolio = np.random.rand(len(self.currencies), 1)
        summation = np.sum(self.portfolio)
        self.portfolio = np.divide(self.portfolio, summation)
        self.portfolio = np.round(self.portfolio, 2)
        for i in range(self.timeframe):
            self.timestep()
        
    def reallocate(self, currencyAllocation):
        exchange = np.subtract(currencyAllocation, self.portfolio)
        oldPortfolio = self.portfolio.copy()
        while (exchange == 0.0).all() != True:
            minIndex = np.unravel_index(np.argmin(exchange, axis=None), exchange.shape)
            maxIndex = np.unravel_index(np.argmax(exchange, axis=None), exchange.shape)
            self.exchangeCurrencies(minIndex, maxIndex, min(abs(exchange[minIndex]), exchange[maxIndex]))
            saveValue = exchange[minIndex]
            exchange[minIndex] = exchange[maxIndex] + exchange[minIndex]
            exchange[maxIndex] = saveValue + exchange[maxIndex]
            exchange = np.round(exchange, 2)

    def incrementTime(self):
        currentTime = str(self.getCurrentTime())
        currentIndex = self.df[self.majorPairs[0]].loc[self.df[self.majorPairs[0]]['Timestamp'] 
                                                       == currentTime].index.values[0]
        self.time = str(self.df[self.majorPairs[0]]['Timestamp'].loc[currentIndex + 1])
        
    def timestep(self):
        if self.time is None:
            for pair in self.majorPairs:
                if pair in self.df.keys():
                    self.time = self.df[pair]['Timestamp'].loc[0]
        else:
            self.incrementTime()
    
    def getCurrentTime(self):
        return self.time
    
    def calculateValue(self):
        totalValue = self.value
        priceChange = self.getRates()
        newValue = np.sum(np.multiply(self.portfolio, priceChange))
        self.value *= newValue
        
    def exchangeCurrencies(self, source, target, amount):
        self.portfolio[source] -= amount
        self.portfolio[target] += amount
        self.value -= amount * self.fee * self.value
    
    def getRates(self):
        rates = np.zeros(shape=(len(self.currencies), 1))
        rates[0,0] = 1
        for i in range(1, len(self.currencies)):
            current = self.currencies[i]
            pair = 'USD' + current
            if pair in self.df.keys():
                index = self.df[pair].loc[self.df[pair]['Timestamp'] 
                                     == self.getCurrentTime()].index.values[0]
                initialPrice = 1/self.df[pair].loc[index - 1, 'Open']
                finalPrice = 1/self.df[pair].loc[index, 'Open']
                rates[i, 0] = finalPrice/initialPrice
            else:
                pair = current + 'USD'
                index = self.df[pair].loc[self.df[pair]['Timestamp'] 
                                     == self.getCurrentTime()].index.values[0]
                initialPrice = self.df[pair].loc[index - 1, 'Open']
                finalPrice = self.df[pair].loc[index, 'Open']
                rates[i, 0] = finalPrice/initialPrice
        return rates
    def processTimeFrame(self, timePeriod):
        priceMatrix = np.zeros(shape=(len(self.currencies), timePeriod, 3))
        currentTime = self.getCurrentTime()
        dimensions = ['Open', 'High', 'Low']
        dimension = -1
        absoluteValue = 0
        for dimensionName in dimensions:
            dimension += 1
            m = 0
            for currency in self.currencies:
                print(currency)
                if currency == self.reference:
                    priceMatrix[m, :, dimension] = 1
                else:
                    if currency + 'USD' in self.df.keys():
                        pair = currency + 'USD'
                    elif 'USD' + currency in self.df.keys():
                        pair = 'USD' + currency
                    else:
                        continue
                    i = self.df[pair].loc[self.df[pair]['Timestamp'] 
                                 == self.getCurrentTime()].index.values[0]
                    openValues = self.df[pair].loc[i-timePeriod+1:i, dimensionName].values
                    if dimensionName == 'Open':
                        absoluteValue = openValues[-1]
                    openProcessed = openValues / absoluteValue
                    #print(openProcessed)
                    priceMatrix[m, :, dimension] = openProcessed
                m += 1
        print(priceMatrix)

In [88]:
test = Market(['EUR','USD'], os.path.abspath('../Data_Processing/ProcessedData'))

In [89]:
#Test

test.initPortfolio()
test.processTimeFrame(5)
# test.calculateValue()
# reallocation = np.array([0.7, 0.3])[np.newaxis]
# test.reallocate(reallocation.T)
#print(test.df['EURUSD'].loc[50:100, ['Open']].values)

USD
EUR
USD
EUR
USD
EUR
[[[1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]]

 [[1.00010461 1.00015215 1.0000951 ]
  [1.00008559 1.00011412 1.00007608]
  [0.99995245 1.00012363 0.99995245]
  [0.99994294 1.00002853 0.99992392]
  [1.         1.00000951 0.99996196]]]
